# BBLF AI Selector: Part 2: During Tournament Optimal Squad - Optimser (Round 3)

# 0. Prerequistes

In [1]:
# 0. Prerequistes

import pandas as pd
import numpy as np
import os
import random
from mip import Model, xsum, maximize, BINARY 

os.getcwd()
directory = 'C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI'
add_data_directory = 'C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI/data/add_data_created/round_3'
over_data_directory = 'C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI/data/add_data_created/overall'
py_data_directory = 'C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI/data/python_datasets'

# 1. Data Extraction 

In [2]:
# 1. Data Extraction 
    # Current Round & Optimal Number of Rounds
current_round = 3
opt_round = 3

    # Pull in player price data csv file 

price_df = pd.read_csv(os.path.join(add_data_directory,'player_price_rnd_3.csv'), low_memory=False)
#price_df = pd.read_csv(os.path.join(add_data_directory,'player_price_rnd_', str(current_round), '.csv'), low_memory=False)

print(price_df) 

    # Player Role Flags

price_df["Wk_f"] = np.where((price_df["Role"] == "WK"), 1, 0)
price_df["Bat_f"] = np.where((price_df["Role"] == "WK") |(price_df["Role"] == "BAT") | (price_df["Role"] == "ALLR") , 1, 0)
price_df["Bowl_f"] = np.where((price_df["Role"] == "BOWL") | (price_df["Role"] == "ALLR") , 1, 0)
price_df = price_df[["Full_Name", "Price", "Team","Wk_f", "Bat_f", "Bowl_f", "Role", "Available", "In_Team"]].rename(columns = {"Full_Name": "Name"}) 
    
    # Pull in team fixture csv file

team_fix_df = pd.read_csv(os.path.join(over_data_directory,'team_loc_fixture.csv'), low_memory=False)
team_fix_df = team_fix_df[team_fix_df.Round >= current_round].dropna()
team_fix_df = team_fix_df[team_fix_df.Round <= current_round + opt_round - 1].dropna()
print(team_fix_df)

    # Pull player expected points csv file
exp_pts_df = pd.read_csv(os.path.join(py_data_directory,'bbl14_fullteam_model_score_during_rnd_3.csv'), low_memory=False).drop(["Unnamed: 0", "player"], axis = 1)
print(exp_pts_df)

    # Join team fixture to player price to create a row for each game
game_df = pd.merge(price_df , team_fix_df, left_on = ["Team"], right_on = ["Team"], how = "left")
print(game_df)

    # Join on expected points for each game
print(game_df.columns)
print(exp_pts_df.columns)
player_df_raw = pd.merge(game_df, exp_pts_df, left_on = ["Name", "Team", "Opposition", "Venue", "Home_f","Round"], right_on = ["Name", "Team", "opp", "venue", "Home_f","Round"], how = "left")
player_df_raw["exp_pts"] = np.where((player_df_raw["Role"] == "WK"), player_df_raw["exp_pts"] + 10.51, player_df_raw["exp_pts"] + 4.53)
player_df_raw["weight"] = 1
print(player_df_raw)

    # Aggregate by player name (calculate total expected points for each player for x number of rounds)
player_df = player_df_raw.groupby(['Name', 'Price', "Team", "Wk_f", "Bat_f", "Bowl_f", "Role", "weight","Available", "In_Team"], as_index=False).agg(
exp_pts=('exp_pts',"sum"))
print(player_df)

player_df_next_rnd = player_df_raw[player_df_raw.Round == current_round].groupby(['Name', 'Price', "Team", "Wk_f", "Bat_f", "Bowl_f", "Role", "weight","Available", "In_Team"], as_index=False).agg(
next_rnd_exp_pts=('exp_pts',"sum"))
player_df_next_rnd = player_df_next_rnd[["Name", "Team", "next_rnd_exp_pts"]]


      Last_Name First_Name First_Name_Short        Full_Name         player  \
0        Abbott       Sean               SA      Sean Abbott      SA Abbott   
1          Agar     Ashton               AC      Ashton Agar        AC Agar   
2          Agar        Wes               WA         Wes Agar        WA Agar   
3         Allen     Fabian              NaN     Fabian Allen   Fabian Allen   
4         Allen       Finn              NaN       Finn Allen     Finn Allen   
..          ...        ...              ...              ...            ...   
142       Wells        Jon               JW        Jon Wells       JW Wells   
143  Wildermuth       Jack               JD  Jack Wildermuth  JD Wildermuth   
144        Wood       Jack               JP        Jack Wood        JP Wood   
145      Wright        Mac                M       Mac Wright       M Wright   
146       Zampa       Adam                A       Adam Zampa        A Zampa   

      Price                 Team  Role  Available  

# 2. Optimisation Process (Round 3) - Normal 3 Trades

Optimisation Objective: Maximise the number of expected fantasy points

Constraints: 
1. Number of players selected must be 12
2. Atleast 1 wicketkeeper
3. Atleast 6 batters
4. Atleast 5 bowlers
5. Total budget of team is less than $1,762,800
6. Atleast 9 players in current team (3 trades)

In [6]:
# Optimisation Setup

points = player_df["exp_pts"]

price = player_df["Price"]

#model_df.loc[:,"opp_Adelaide Strikers":"venue_WACA"] = model_df.loc[:,"opp_Adelaide Strikers":"venue_WACA"].astype(object)

weight = player_df["weight"]
available = player_df["Available"]
in_team = player_df["In_Team"]
wk_weight = player_df["Wk_f"]
bat_weight = player_df["Bat_f"]
bowl_weight = player_df["Bowl_f"]

play_cnt, total_player = 12, range(len(price))
team_play_cnt, total_team_player = 9, range(len(price))
wk_cnt, total_wk = 1, range(len(price))
bat_cnt, total_bat = 6, range(len(price))
bowl_cnt, total_bowl = 5, range(len(price))
budget, total_budget = 1767100, range(len(price))

m = Model("knapsack")


x = [m.add_var(var_type=BINARY) for i in total_player]
print(x)

m.objective = maximize(xsum(points[i]*x[i] for i in total_player))

m += xsum(weight[i] * x[i] for i in total_player) == play_cnt
m += xsum(wk_weight[i] * x[i] for i in total_wk) >= wk_cnt
m += xsum(bat_weight[i] * x[i] for i in total_bat) >= bat_cnt
m += xsum(bowl_weight[i] * x[i] for i in total_bowl) >= bowl_cnt
m += xsum(price[i] * x[i] for i in total_budget) <= budget
m += xsum(available[i] * x[i] for i in total_player) == play_cnt
m += xsum(in_team[i] * x[i] for i in total_team_player) >= team_play_cnt

m.optimize()

selected = [i for i in total_player if x[i].x >= 0.99]
print("selected items: {}".format(selected))

sel_player_df = player_df.iloc[selected]
sel_player_df = pd.merge(sel_player_df, player_df_next_rnd, left_on = ["Name","Team"], right_on = ["Name","Team"], how = "left")
print("Total Expect Points:", sum(sel_player_df["exp_pts"]))
print("Total Next Round Points:", sum(sel_player_df["next_rnd_exp_pts"]))
print("Total Team Cost:", sum(sel_player_df["Price"]))
print("Number of Wk:", sum(sel_player_df["Wk_f"]))
print("Number of Bat:", sum(sel_player_df["Bat_f"]))
print("Number of Bowl:", sum(sel_player_df["Bowl_f"]))
print("Available Players:", sum(sel_player_df["Available"]))
print("Current Players Remaining:", sum(sel_player_df["In_Team"]))

print(sel_player_df.sort_values(by = "exp_pts", ascending = False))
print(sel_player_df[sel_player_df['In_Team'] == 1])
print(sel_player_df[sel_player_df['In_Team'] == 0])


sel_player_df.to_csv('C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI/data/python_datasets/optimal_round_3.csv')


[<mip.entities.Var object at 0x00000258C378F0A0>, <mip.entities.Var object at 0x00000258C378E530>, <mip.entities.Var object at 0x00000258C378E4D0>, <mip.entities.Var object at 0x00000258C378E350>, <mip.entities.Var object at 0x00000258C378F550>, <mip.entities.Var object at 0x00000258C378D510>, <mip.entities.Var object at 0x00000258C378E110>, <mip.entities.Var object at 0x00000258C378D930>, <mip.entities.Var object at 0x00000258C378F5B0>, <mip.entities.Var object at 0x00000258C378E770>, <mip.entities.Var object at 0x00000258C378E890>, <mip.entities.Var object at 0x00000258C378EB00>, <mip.entities.Var object at 0x00000258C378DEA0>, <mip.entities.Var object at 0x00000258C378E5F0>, <mip.entities.Var object at 0x00000258C378E7D0>, <mip.entities.Var object at 0x00000258C378F430>, <mip.entities.Var object at 0x00000258C378E950>, <mip.entities.Var object at 0x00000258C378E320>, <mip.entities.Var object at 0x00000258C378EEC0>, <mip.entities.Var object at 0x00000258C378EBC0>, <mip.entities.Var o

# 3. Optimisation Process (Round 3) - Normal 3 Trades + Boost

Optimisation Objective: Maximise the number of expected fantasy points

Constraints: 
1. Number of players selected must be 12
2. Atleast 1 wicketkeeper
3. Atleast 6 batters
4. Atleast 5 bowlers
5. Total budget of team is less than $1,762,800
6. Atleast 8 players in current team (4 trades)

In [4]:
# Optimisation Setup

points = player_df["exp_pts"]

price = player_df["Price"]

#model_df.loc[:,"opp_Adelaide Strikers":"venue_WACA"] = model_df.loc[:,"opp_Adelaide Strikers":"venue_WACA"].astype(object)

weight = player_df["weight"]
available = player_df["Available"]
in_team = player_df["In_Team"]
wk_weight = player_df["Wk_f"]
bat_weight = player_df["Bat_f"]
bowl_weight = player_df["Bowl_f"]

play_cnt, total_player = 12, range(len(price))
team_play_cnt, total_team_player = 8, range(len(price))
wk_cnt, total_wk = 1, range(len(price))
bat_cnt, total_bat = 6, range(len(price))
bowl_cnt, total_bowl = 5, range(len(price))
budget, total_budget = 1767100, range(len(price))

m = Model("knapsack")


x = [m.add_var(var_type=BINARY) for i in total_player]
print(x)

m.objective = maximize(xsum(points[i]*x[i] for i in total_player))

m += xsum(weight[i] * x[i] for i in total_player) == play_cnt
m += xsum(wk_weight[i] * x[i] for i in total_wk) >= wk_cnt
m += xsum(bat_weight[i] * x[i] for i in total_bat) >= bat_cnt
m += xsum(bowl_weight[i] * x[i] for i in total_bowl) >= bowl_cnt
m += xsum(price[i] * x[i] for i in total_budget) <= budget
m += xsum(available[i] * x[i] for i in total_player) == play_cnt
m += xsum(in_team[i] * x[i] for i in total_team_player) >= team_play_cnt

m.optimize()

selected = [i for i in total_player if x[i].x >= 0.99]
print("selected items: {}".format(selected))

sel_player_df = player_df.iloc[selected]
sel_player_df = pd.merge(sel_player_df, player_df_next_rnd, left_on = ["Name","Team"], right_on = ["Name","Team"], how = "left")
print("Total Expect Points:", sum(sel_player_df["exp_pts"]))
print("Total Next Round Points:", sum(sel_player_df["next_rnd_exp_pts"]))
print("Total Team Cost:", sum(sel_player_df["Price"]))
print("Number of Wk:", sum(sel_player_df["Wk_f"]))
print("Number of Bat:", sum(sel_player_df["Bat_f"]))
print("Number of Bowl:", sum(sel_player_df["Bowl_f"]))
print("Available Players:", sum(sel_player_df["Available"]))
print("Current Players Remaining:", sum(sel_player_df["In_Team"]))

print(sel_player_df.sort_values(by = "exp_pts", ascending = False))
print(sel_player_df[sel_player_df['In_Team'] == 1])
print(sel_player_df[sel_player_df['In_Team'] == 0])


sel_player_df.to_csv('C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI/data/python_datasets/optimal_round_3_boost.csv')


[<mip.entities.Var object at 0x00000258C372A980>, <mip.entities.Var object at 0x00000258C3729E40>, <mip.entities.Var object at 0x00000258C3729E70>, <mip.entities.Var object at 0x00000258C372B010>, <mip.entities.Var object at 0x00000258C372AC50>, <mip.entities.Var object at 0x00000258C372A050>, <mip.entities.Var object at 0x00000258C372A590>, <mip.entities.Var object at 0x00000258C372A620>, <mip.entities.Var object at 0x00000258C372A650>, <mip.entities.Var object at 0x00000258C372AB00>, <mip.entities.Var object at 0x00000258C372A080>, <mip.entities.Var object at 0x00000258C269D720>, <mip.entities.Var object at 0x00000258C1E97880>, <mip.entities.Var object at 0x00000258932806A0>, <mip.entities.Var object at 0x00000258932805B0>, <mip.entities.Var object at 0x0000025893280220>, <mip.entities.Var object at 0x0000025893282440>, <mip.entities.Var object at 0x00000258C372AAA0>, <mip.entities.Var object at 0x00000258C372A3B0>, <mip.entities.Var object at 0x00000258C372A410>, <mip.entities.Var o

# 3. Optimisation Process (Round 3) - Normal 3 Trades + Sam Konstas (Boost)

Optimisation Objective: Maximise the number of expected fantasy points

Constraints: 
1. Number of players selected must be 12
2. Atleast 1 wicketkeeper
3. Atleast 6 batters
4. Atleast 5 bowlers
5. Total budget of team is less than $1,798,900
6. Atleast 9 players in current team (3 trades)

In [5]:
# Optimisation Setup

points = player_df["exp_pts"]

price = player_df["Price"]

#model_df.loc[:,"opp_Adelaide Strikers":"venue_WACA"] = model_df.loc[:,"opp_Adelaide Strikers":"venue_WACA"].astype(object)

weight = player_df["weight"]
available = player_df["Available"]
in_team = player_df["In_Team"]
wk_weight = player_df["Wk_f"]
bat_weight = player_df["Bat_f"]
bowl_weight = player_df["Bowl_f"]

play_cnt, total_player = 12, range(len(price))
team_play_cnt, total_team_player = 9, range(len(price))
wk_cnt, total_wk = 1, range(len(price))
bat_cnt, total_bat = 6, range(len(price))
bowl_cnt, total_bowl = 5, range(len(price))
budget, total_budget = 1798900, range(len(price))

m = Model("knapsack")


x = [m.add_var(var_type=BINARY) for i in total_player]
print(x)

m.objective = maximize(xsum(points[i]*x[i] for i in total_player))

m += xsum(weight[i] * x[i] for i in total_player) == play_cnt
m += xsum(wk_weight[i] * x[i] for i in total_wk) >= wk_cnt
m += xsum(bat_weight[i] * x[i] for i in total_bat) >= bat_cnt
m += xsum(bowl_weight[i] * x[i] for i in total_bowl) >= bowl_cnt
m += xsum(price[i] * x[i] for i in total_budget) <= budget
m += xsum(available[i] * x[i] for i in total_player) == play_cnt
m += xsum(in_team[i] * x[i] for i in total_team_player) >= team_play_cnt

m.optimize()

selected = [i for i in total_player if x[i].x >= 0.99]
print("selected items: {}".format(selected))

sel_player_df = player_df.iloc[selected]
sel_player_df = pd.merge(sel_player_df, player_df_next_rnd, left_on = ["Name","Team"], right_on = ["Name","Team"], how = "left")
print("Total Expect Points:", sum(sel_player_df["exp_pts"]))
print("Total Next Round Points:", sum(sel_player_df["next_rnd_exp_pts"]))
print("Total Team Cost:", sum(sel_player_df["Price"]))
print("Number of Wk:", sum(sel_player_df["Wk_f"]))
print("Number of Bat:", sum(sel_player_df["Bat_f"]))
print("Number of Bowl:", sum(sel_player_df["Bowl_f"]))
print("Available Players:", sum(sel_player_df["Available"]))
print("Current Players Remaining:", sum(sel_player_df["In_Team"]))

print(sel_player_df.sort_values(by = "exp_pts", ascending = False))
print(sel_player_df[sel_player_df['In_Team'] == 1])
print(sel_player_df[sel_player_df['In_Team'] == 0])


sel_player_df.to_csv('C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI/data/python_datasets/optimal_round_3_boost_SK.csv')


[<mip.entities.Var object at 0x00000258C378EE90>, <mip.entities.Var object at 0x00000258C378EFB0>, <mip.entities.Var object at 0x00000258C378EFE0>, <mip.entities.Var object at 0x00000258C378C310>, <mip.entities.Var object at 0x00000258C378C3D0>, <mip.entities.Var object at 0x00000258C378C730>, <mip.entities.Var object at 0x00000258C378C820>, <mip.entities.Var object at 0x00000258C378D0C0>, <mip.entities.Var object at 0x00000258C378C1F0>, <mip.entities.Var object at 0x00000258C378CA00>, <mip.entities.Var object at 0x00000258C378CFA0>, <mip.entities.Var object at 0x00000258C378CEE0>, <mip.entities.Var object at 0x00000258C378C940>, <mip.entities.Var object at 0x00000258C378D900>, <mip.entities.Var object at 0x00000258C378CF40>, <mip.entities.Var object at 0x00000258C378C910>, <mip.entities.Var object at 0x00000258C378C190>, <mip.entities.Var object at 0x00000258C378C2E0>, <mip.entities.Var object at 0x00000258C378CAC0>, <mip.entities.Var object at 0x00000258C378C5E0>, <mip.entities.Var o